In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio
import os
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# IMPORTANT: Remember to create a .env variable containing: OPENAI_API_KEY=sk-xyz where xyz is your key

# Access the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key

nest_asyncio.apply()

In [2]:
import openai
import numpy as np
import pandas as pd
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain,RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import UnstructuredFileLoader
from langchain.prompts import PromptTemplate

In [3]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Milvus
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/Robinson_Advisory.txt", encoding="windows-1252")
index = VectorstoreIndexCreator().from_loaders([loader])


text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=0
)
documents = loader.load_and_split()
# texts = text_splitter.split_text(document)
docs = text_splitter.split_documents(documents)

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
embeddings = OpenAIEmbeddings()

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="contract",
    connection_args={"host": "192.168.137.236", "port": "19530", "database": "contract"},
)
vector_db = Milvus(
    embeddings,
    connection_args={"host": "192.168.137.236", "port": "19530"},
    collection_name="contract",
)

In [5]:
prompt_template = '''
You are a contract advisor expert with immense knowledge and experience in the field.
Answer my questions based on your knowledge and our older conversation. Do not make up answers.
If you do not know the answer to a question, just say "I don't know".

{context}

Given the following conversation and a follow up question, answer the question.

{chat_history}

question: {question}
'''

PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "chat_history", "question"]
        )

In [13]:
from langchain.memory import ConversationBufferMemory
# memory = ConversationBufferMemory(memory_key="history", input_key="query", output_key='answer', return_messages=True)
# from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
# memory = ConversationBufferWindowMemory(memory_key="history", k=2)   

memory = ConversationBufferWindowMemory(
                                    k=2,
                                    memory_key="chat_history",
                                    max_len=50,
                                    return_messages=True,
                                    output_key='answer'
                                )

llm = ChatOpenAI(temperature=0)
chain = RetrievalQAWithSourcesChain.from_chain_type(
                        llm,
                        chain_type="map_rerank",
                        retriever=vector_db.as_retriever(),
                        memory=memory,
                    )

qa = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=vector_db.as_retriever(),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": PROMPT,    
     },
    memory=memory
 )
# chain_type_kwargs = {'prompt': PROMPT}

# chain = RetrievalQA.from_chain_type(                             
#                                 llm,
#                                 chain_type="stuff",
#                                 retriever=vector_db.as_retriever(),
#                                 chain_type_kwargs=chain_type_kwargs,
#                                 memory=memory
#                             )


In [14]:
# testing the responses
query = "What are the payments to the Advisor under the Agreement?"
chain(PROMPT.format(question=query, chat_history=memory.chat_memory.messages, context=vector_db.similarity_search_with_score(query)))

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'question': '\nYou are a contract advisor expert with immense knowledge and experience in the field.\nAnswer my questions based on your knowledge and our older conversation. Do not make up answers.\nIf you do not know the answer to a question, just say "I don\'t know".\n\n[(Document(page_content=\'6. Compensation:\\n6.1 As full and sole consideration for the Services, and subject to the performance of the Services, the Company shall pay the Advisor, against an invoice provided to the Company by Advisor, hourly fees at a rate of USD 9 (nine) per Billable Hour as defined below, limited to a maximum of USD 1,500 per month (the "Fees"). In addition, the Company shall pay the advisor USD 100 per month to finance a workspace for the Advisor, as long as the Advisor actually hires a professional workspace (the “Workspace Expense”). Advisor will not be entitled to any additional fees or expense reimbursement whatsoever, except as expressly provided for in this Agreement. \\nBillable Hour: Net 

In [17]:
chain({"question": "Who are the parties to the Agreement and what are their defined names?"})


/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'question': 'Who are the parties to the Agreement and what are their defined names?',
 'chat_history': [HumanMessage(content='\nYou are a contract advisor expert with immense knowledge and experience in the field.\nAnswer my questions based on your knowledge and our older conversation. Do not make up answers.\nIf you do not know the answer to a question, just say "I don\'t know".\n\n[(Document(page_content=\'6. Compensation:\\n6.1 As full and sole consideration for the Services, and subject to the performance of the Services, the Company shall pay the Advisor, against an invoice provided to the Company by Advisor, hourly fees at a rate of USD 9 (nine) per Billable Hour as defined below, limited to a maximum of USD 1,500 per month (the "Fees"). In addition, the Company shall pay the advisor USD 100 per month to finance a workspace for the Advisor, as long as the Advisor actually hires a professional workspace (the “Workspace Expense”). Advisor will not be entitled to any additional fee